In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT  user_id,
        lt_day,
       CASE 
           WHEN lt_day<=365 THEN 'новый'
           WHEN lt_day>365 THEN 'не новый'
           END as is_new,
        age,
       CASE 
           WHEN gender_segment=0 THEN 'мужчина'
           WHEN gender_segment=1 THEN 'женщина'
           END as gender_segment,
        os_name,
        cpe_type_name,
        country,
        city,
        a.title as age_segment,
        t.title as traffic_segment,
        lt.title as lifetime_segment,
        nps_score,
       CASE 
           WHEN nps_score<=6 THEN 'критики'
           WHEN nps_score>8 THEN 'cторонники'
           WHEN nps_score>6 and nps_score<9  THEN 'нейтралы'
           END as nps_group
FROM user as u
LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
LEFT OUTER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT OUTER JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.sample(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
388200,U37FG0,116,новый,45.0,мужчина,ANDROID,TABLET,Россия,НижнийНовгород,05 45-54,04 1-5,04 4-6,10,cторонники
322051,QNXZ4Q,151,новый,40.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,04 35-44,04 1-5,04 4-6,10,cторонники
124418,GEO8N9,1114,не новый,27.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,03 25-34,07 15-20,08 36+,1,критики
161126,IBJ55Q,76,новый,28.0,мужчина,ANDROID,SMARTPHONE,Россия,Тверь,03 25-34,04 1-5,03 3,10,cторонники
65644,DDQRHX,1076,не новый,52.0,мужчина,ANDROID,TABLET,Россия,Москва,05 45-54,03 0.1-1,07 25-36,7,нейтралы
193534,K025VK,1160,не новый,42.0,женщина,ANDROID,SMARTPHONE,Россия,НижнийНовгород,04 35-44,04 1-5,08 36+,10,cторонники
19555,B0A5NC,578,не новый,29.0,мужчина,IOS,SMARTPHONE,Россия,Москва,03 25-34,07 15-20,06 13-24,9,cторонники
465129,Y2EY8G,1682,не новый,39.0,мужчина,ANDROID,SMARTPHONE,Россия,Астрахань,04 35-44,09 25-30,08 36+,10,cторонники
297102,PDAY3W,3235,не новый,57.0,женщина,ANDROID,SMARTPHONE,Россия,Саратов,06 55-64,05 5-10,08 36+,5,критики
75573,DW7DN8,198,новый,36.0,мужчина,ANDROID,SMARTPHONE,Россия,Новосибирск,04 35-44,07 15-20,05 7-12,10,cторонники


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


Всего наблюдений - 52493. Есть пропуски в данных о возрасте и поле.
Т.к. в анализе будут использованы возрастные категории, определяемые полем age_segment, которое взято из отдельной таблицы, не будем заполнять пропуски в поле age.
Пропуски в данных о поле заполнить не представляется возможным.

In [6]:
df['age_segment'].unique()

array(['05 45-54', '06 55-64', '04 35-44', '02 16-24', '03 25-34',
       '07 66 +', '01 до 16', '08 n/a'], dtype=object)

In [7]:
df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


In [8]:
df['lifetime_segment'].unique()

array(['08 36+', '06 13-24', '05 7-12', '04 4-6', '07 25-36', '02 2',
       '03 3', '01 1'], dtype=object)

Есть аномальное значение количества дней жизни -21, но т.к. поле lt_day используется только для определения категории клиента новый/не новый, а при такой аномалии категория определится верно как "новый", наличие данной аномалии не повлияет на построение дашбордов.


Ссылка на дашборд:
https://public.tableau.com/views/telecom_17144126741380/NPS_8?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

Выводы в соответствии с условием задачи размещены на дашборде.